In [ ]:

#En teória, se hizo para ayudar, esperemos que así sea por los siglos de los siglos
#Por: Francisco Armando Sánchez Díaz (202355940) El presente es de uso libre para con quién lo solicite

In [ ]:
import requests
import pandas as pd
import os
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

BASE_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# ─────────────────────────────────────────────
# CLASIFICACIÓN DEL IDENTIFICADOR
# ─────────────────────────────────────────────

def clasificar_termino(termino):
    """
    Detecta si el término es:
    - Un ID numérico de gen (solo dígitos)
    - Un accession de proteína (patrón tipo NP_XXXXXX, XP_XXXXXX, P12345, etc.)
    - Un nombre de gen/proteína para buscar
    """
    termino = termino.strip()

    if termino.isdigit():
        return "gene_id", termino

    if re.match(r'^[NXYWA]P_\d+(\.\d+)?$', termino, re.IGNORECASE):
        return "protein_accession", termino


    if re.match(r'^[PQO]\d{5}(\.\d+)?$', termino, re.IGNORECASE):
        return "protein_accession", termino


    if re.match(r'^[A-Z]{2}_\d+', termino, re.IGNORECASE):
        return "protein_accession", termino


    return "nombre", termino


In [ ]:
# ─────────────────────────────────────────────
# BÚSQUEDA POR NOMBRE :3
# ─────────────────────────────────────────────

def buscar_id_gen(nombre):
    """Busca el Gene ID en NCBI Gene dado un nombre de gen o proteína."""
    params = {
        "db": "gene",
        "term": f"({nombre}[Gene Name] OR {nombre}[Protein Name]) AND Homo sapiens[Organism]",
        "retmode": "json",
        "retmax": 1
    }
    r = requests.get(BASE_URL + "esearch.fcgi", params=params)
    if r.status_code != 200:
        return None
    ids = r.json().get("esearchresult", {}).get("idlist", [])
    return ids[0] if ids else None


def buscar_id_proteina(nombre):
    """Busca el accession de proteína en NCBI Protein dado un nombre."""
    params = {
        "db": "protein",
        "term": f"{nombre}[Protein Name] AND Homo sapiens[Organism] AND refseq[Filter]",
        "retmode": "json",
        "retmax": 1
    }
    r = requests.get(BASE_URL + "esearch.fcgi", params=params)
    if r.status_code != 200:
        return None
    ids = r.json().get("esearchresult", {}).get("idlist", [])
    return ids[0] if ids else None

In [ ]:
# ─────────────────────────────────────────────
# OBTENCIÓN DE DATOS DE GEN
# ─────────────────────────────────────────────

def obtener_datos_gen(gene_id):
    """Usa ESummary + EFetch para obtener información completa de un gen."""
    datos = {"Tipo": "Gen", "ID Búsqueda": gene_id}


    r = requests.get(BASE_URL + "esummary.fcgi", params={
        "db": "gene", "id": gene_id, "retmode": "json"
    })
    if r.status_code == 200:
        doc = r.json().get("result", {}).get(str(gene_id), {})
        datos["Símbolo oficial"]         = doc.get("name", "No disponible")
        datos["Nombre oficial"]          = doc.get("description", "No disponible")
        datos["Otros nombres (alias)"]   = ", ".join(doc.get("otheraliases", "").split("|")) if doc.get("otheraliases") else "No disponible"
        datos["Especie"]                 = doc.get("organism", {}).get("scientificname", "No disponible")
        datos["ID de gen"]               = doc.get("uid", gene_id)
        datos["Tipo de gen"]             = doc.get("genetype", "No disponible")
        datos["Resumen"]                 = doc.get("summary", "No disponible")[:500] if doc.get("summary") else "No disponible"
        datos["Estado"]                  = doc.get("status", "No disponible")


        loc = doc.get("locationhist", [{}])
        if loc:
            l = loc[0]
            datos["Cromosoma"]           = l.get("chraccver", doc.get("chromosome", "No disponible"))
            datos["Brazo cromosómico"]   = l.get("chrstart", "No disponible")
            datos["Localización NCBI"]   = doc.get("maplocation", "No disponible")


        genomic = doc.get("genomicinfo", [{}])
        if genomic:
            g = genomic[0]
            start = g.get("chrstart", None)
            stop  = g.get("chrstop", None)
            datos["Posición inicio (bp)"] = start if start is not None else "No disponible"
            datos["Posición fin (bp)"]    = stop  if stop  is not None else "No disponible"
            if start is not None and stop is not None:
                datos["Tamaño del gen (bp)"] = abs(int(stop) - int(start)) + 1
            else:
                datos["Tamaño del gen (bp)"] = "No disponible"
            datos["Exones anotados"]     = g.get("exoncount", "No disponible")


        datos["ARNm de referencia (NM_)"] = doc.get("mim", "No disponible")

    time.sleep(0.4)


    r2 = requests.get(BASE_URL + "efetch.fcgi", params={
        "db": "gene", "id": gene_id, "rettype": "gene_table", "retmode": "text"
    })
    if r2.status_code == 200:
        txt = r2.text

        if datos.get("Exones anotados") in (None, "No disponible"):
            m = re.search(r",\s*(\d+)\s+exons", txt)
            datos["Exones anotados"] = m.group(1) if m else "No disponible"


        m = re.search(r"NP_\d+\.\d+", txt)
        datos["Proteína de referencia (NP_)"] = m.group(0) if m else "No disponible"


        m = re.search(r"NM_\d+\.\d+", txt)
        if m:
            datos["ARNm de referencia (NM_)"] = m.group(0)


    datos["Enlace NCBI Gene"] = f"https://www.ncbi.nlm.nih.gov/gene/{gene_id}"

    return datos


In [ ]:
# ─────────────────────────────────────────────
# OBTENCIÓN DE DATOS DE PROTEÍNA
# ─────────────────────────────────────────────

def obtener_datos_proteina_por_accession(accession):
    """Obtiene datos de proteína desde NCBI Protein usando el accession."""
    datos = {"Tipo": "Proteína", "ID Búsqueda": accession}


    if not accession.isdigit():
        r0 = requests.get(BASE_URL + "esearch.fcgi", params={
            "db": "protein", "term": accession, "retmode": "json", "retmax": 1
        })
        if r0.status_code == 200:
            ids = r0.json().get("esearchresult", {}).get("idlist", [])
            uid = ids[0] if ids else None
        else:
            uid = None
    else:
        uid = accession

    if not uid:
        datos["Error"] = "No se encontró el accession en NCBI Protein"
        return datos

    time.sleep(0.4)


    r = requests.get(BASE_URL + "esummary.fcgi", params={
        "db": "protein", "id": uid, "retmode": "json"
    })
    if r.status_code == 200:
        doc = r.json().get("result", {}).get(str(uid), {})
        datos["Nombre de la proteína"]     = doc.get("title", "No disponible")
        datos["Accession"]                 = doc.get("accessionversion", accession)
        datos["Longitud (aa)"]             = doc.get("slen", "No disponible")
        datos["Organismo"]                 = doc.get("organism", "No disponible")
        datos["Fecha de creación"]         = doc.get("createdate", "No disponible")
        datos["Fecha de actualización"]    = doc.get("updatedate", "No disponible")
        datos["Estado"]                    = doc.get("status", "No disponible")
        datos["UID NCBI"]                  = uid
        datos["Enlace NCBI Protein"]       = f"https://www.ncbi.nlm.nih.gov/protein/{accession}"

    time.sleep(0.4)


    r2 = requests.get(BASE_URL + "efetch.fcgi", params={
        "db": "protein", "id": uid, "rettype": "gp", "retmode": "text"
    })
    if r2.status_code == 200:
        gp = r2.text


        m = re.search(r'/gene="([^"]+)"', gp)
        datos["Gen que codifica"]          = m.group(1) if m else "No disponible"


        m = re.search(r'GeneID:(\d+)', gp)
        datos["Gene ID (NCBI)"]            = m.group(1) if m else "No disponible"


        m = re.search(r'/product="([^"]+)"', gp)
        datos["Producto anotado"]          = m.group(1) if m else "No disponible"


        m = re.search(r'/note="([^"]+)"', gp)
        datos["Nota funcional"]            = m.group(1)[:300] if m else "No disponible"


        m = re.search(r'EC_number="([^"]+)"', gp)
        datos["Número EC"]                 = m.group(1) if m else "No disponible"


        m = re.search(r'coded_by="([^"]+)"', gp)
        datos["ARNm que la codifica"]      = m.group(1) if m else "No disponible"

        m = re.search(r'subcellular_location="([^"]+)"', gp)
        datos["Localización subcelular"]   = m.group(1) if m else "No disponible"


        m = re.search(r'calculated_mol_wt=(\d+)', gp)
        datos["Peso molecular (Da)"]       = m.group(1) if m else "No disponible"


        if datos.get("Gene ID (NCBI)") != "No disponible":
            datos["Enlace NCBI Gen"]       = f"https://www.ncbi.nlm.nih.gov/gene/{datos['Gene ID (NCBI)']}"

    return datos


def obtener_datos_proteina_por_nombre(nombre):
    """Busca proteína por nombre y descarga sus datos."""
    uid = buscar_id_proteina(nombre)
    if not uid:

        gene_id = buscar_id_gen(nombre)
        if gene_id:
            print(f"  → '{nombre}' encontrado como gen (ID: {gene_id}), extrayendo datos de gen...")
            return obtener_datos_gen(gene_id)
        return None
    return obtener_datos_proteina_por_accession(uid)

In [ ]:
def generar_excel(resultados, archivo="informe_genes_proteinas.xlsx"):
    """Genera un Excel con formato profesional y dos hojas (genes / proteínas)."""
    from openpyxl import Workbook
    from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
    from openpyxl.utils import get_column_letter

    genes     = [r for r in resultados if r.get("Tipo") == "Gen"]
    proteinas = [r for r in resultados if r.get("Tipo") == "Proteína"]

    wb = Workbook()

    def escribir_hoja(wb, nombre_hoja, datos, color_cabecera="1F4E79"):
        ws = wb.create_sheet(title=nombre_hoja)
        if not datos:
            ws["A1"] = "Sin datos"
            return

        df = pd.DataFrame(datos)
        columnas = list(df.columns)


        header_font   = Font(name="Arial", bold=True, color="FFFFFF", size=11)
        header_fill   = PatternFill("solid", start_color=color_cabecera)
        header_align  = Alignment(horizontal="center", vertical="center", wrap_text=True)
        thin          = Side(style="thin", color="AAAAAA")
        border        = Border(left=thin, right=thin, top=thin, bottom=thin)

        for col_idx, col_name in enumerate(columnas, 1):
            cell = ws.cell(row=1, column=col_idx, value=col_name)
            cell.font      = header_font
            cell.fill      = header_fill
            cell.alignment = header_align
            cell.border    = border


        alt_fill = PatternFill("solid", start_color="DCE6F1")
        for row_idx, row in enumerate(datos, 2):
            fill = alt_fill if row_idx % 2 == 0 else None
            for col_idx, col_name in enumerate(columnas, 1):
                val  = row.get(col_name, "No disponible")
                cell = ws.cell(row=row_idx, column=col_idx, value=str(val) if val is not None else "No disponible")
                cell.font      = Font(name="Arial", size=10)
                cell.alignment = Alignment(vertical="top", wrap_text=True)
                cell.border    = border
                if fill:
                    cell.fill = fill


        for col_idx, col_name in enumerate(columnas, 1):
            max_len = max(len(str(col_name)), max(
                (len(str(row.get(col_name, ""))) for row in datos), default=0
            ))
            ws.column_dimensions[get_column_letter(col_idx)].width = min(max_len + 4, 50)

        ws.row_dimensions[1].height = 30
        ws.freeze_panes = "A2"


    if genes:
        escribir_hoja(wb, "Genes", genes, "1F4E79")
    if proteinas:
        escribir_hoja(wb, "Proteínas", proteinas, "375623")


    if "Sheet" in wb.sheetnames:
        del wb["Sheet"]

    
    wb.save(archivo)
    print(f"\n✅ Archivo '{archivo}' generado con éxito.")

   
    if IN_COLAB:
       
        files.download(archivo)
    else:
       
        import os
        print(f"📂 Puedes encontrar tu reporte en: {os.path.abspath(archivo)}")